In [0]:
##Cyclic-GAN
# X - paintings and Y - photographs 
# Y' = F(x)
# X' = G(X)

batch_size = int(1)
displaly_size = int(10)


In [0]:
#Install the necessary dependencies
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

In [0]:
#Check for cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assume that we are on a CUDA machine, then this should print a CUDA device:

print(device)

In [0]:
#Mount drive
##To load dataset from Drive
from google.colab import drive
drive.mount('/content/drive')
#Check if drive is mounted
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

In [0]:
import os
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform


class monet_dataset(Dataset):
    """Monet dataset."""

    def __init__(self, path_X, path_Y, transform=None):
        """
        Args:
            path_X (string): Path to the X images directory.
            path_Y (string): Path to the Y images directory.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.Ximages_path = path_X
        self.Yimages_path = path_Y
        self.transform = transform
        self.list_X =  os.listdir(path_X)
        self.list_Y = os.listdir(path_Y)

    def __len__(self):
        return max(len(self.list_X),len(self.list_Y))

    def __getitem__(self, idx):
        img_name_X = os.path.join(self.Ximages_path,
                                self.list_X[idx%len(self.list_X)])
        imageY = io.imread(img_name_X)
        img_name_Y = os.path.join(self.Yimages_path,
                                self.list_Y[idx%len(self.list_Y)])
        imageX = io.imread(img_name_Y)
        
        if self.transform:
            imageX = self.transform(imageX)
            imageY = self.transform(imageY)
        
        return imageX,imageY

In [0]:
from torchvision import transforms, datasets

data_transform = transforms.Compose([
        transforms.ToTensor(),
 #       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
 #       transforms.RandomSizedCrop(224),
 #       transforms.RandomHorizontalFlip(),
 #       transforms.ToTensor(),
 #       transforms.Normalize(mean=[0.485, 0.456, 0.406],
 #                         std=[0.229, 0.224, 0.225]),
    ])

# Create datasets
trainset = dataset('/content/drive/My Drive/Projects/monet2photo/trainA/TrainA','/content/drive/My Drive/Projects/monet2photo/trainB/TrainB', data_transform)
test_set = dataset('/content/drive/My Drive/Projects/monet2photo/testA/TestA','/content/drive/My Drive/Projects/monet2photo/testB/TestB', data_transform)

In [0]:
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np
import torchvision

# functions to show an image
def imshow(img):
    img = img.cpu()
    npimg = img.detach().numpy()
    plt.imshow(torchvision.utils.make_grid(np.transpose(npimg, (1, 2, 0))))
    plt.show()

In [0]:
#Discriminator model 
import torch.nn as nn
import torch.nn.functional as F


class Discriminator(nn.Module):
    def __init__(self):
        
        super(Discriminator, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size = 4, stride= 2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size = 4, stride= 2),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, kernel_size = 4, stride= 2),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, kernel_size = 4, stride= 2),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2),
            nn.Conv2d(512, 1, kernel_size = 14, stride= 1),  #Apply conv to get one dimensional output
        )

    def forward(self, x):
      
        x = self.block(x);
        return x   


In [0]:
#Generator model 


class Generator(nn.Module):
    def __init__(self):
        
        super(Generator, self).__init__()
        
        
        ## input > ENCODER > RESNET nine times > DECODER > output
        
        self.encode = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(3, 64, kernel_size = (7), stride= 1, padding = 2),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size = (3), stride= 2, padding = 0),
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size = (3), stride= 2, padding = 1),
            nn.InstanceNorm2d(256),
            nn.ReLU(),
        )
        
        self.resnet = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size = (1), stride= 1),
            nn.BatchNorm2d(256),            
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size = (3), stride= 1, padding=1),
            nn.BatchNorm2d(256),            
        )
        
        self.decode = nn.Sequential(
            nn.ConvTranspose2d(in_channels = 256, out_channels = 128, kernel_size = 3, stride= 2),      
            nn.InstanceNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(in_channels = 128, out_channels = 64, kernel_size = 2, stride= 2),
            nn.InstanceNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 3, kernel_size = (7), stride= 1),
            nn.InstanceNorm2d(3),
            nn.ReLU(),
        )
        
        
    def forward(self, x):
        x = self.encode(x);
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = F.relu(x + self.resnet(x));
        x = self.decode(x);
        
        return x

In [0]:
#Create an instances of the generator and discriminator

F_gen = Generator()
F_dis = Discriminator()
F_gen = F_gen.cuda()
F_dis = F_dis.cuda()

G_gen = Generator()
G_dis = Discriminator()
G_gen = G_gen.cuda()
G_dis = G_dis.cuda()

In [0]:
real_images_Y = torch.randn(batch_size,3,256,256)
real_images_X = torch.randn(batch_size,3,256,256)

In [0]:
#create optimization function and loss function for both generator and discriminator
import torch.optim as optim

criteriondis = nn.MSELoss()
criterion_feature = nn.L1Loss()
F_gen_optimizer = optim.Adam(F_gen.parameters(), lr=0.0002, betas = (0.5,0.999))
F_dis_optimizer = optim.Adam(F_dis.parameters(), lr = 0.0002, betas = (0.5,0.999))
G_gen_optimizer = optim.Adam(G_gen.parameters(), lr=0.0002, betas = (0.5,0.999))
G_dis_optimizer = optim.Adam(G_dis.parameters(), lr = 0.0002, betas = (0.5,0.999))

In [0]:
#Train the C-GAN
for epoch in range(2):

    F_running_loss_real = 0.0
    F_running_loss_fake = 0.0
    F_running_loss_gen  = 0.0
    G_running_loss_real = 0.0
    G_running_loss_fake = 0.0
    G_running_loss_gen  = 0.0
    F_running_cyclic_loss = 0.0
    G_running_cyclic_loss = 0.0        
    

    for i in range(len(trainset)):
        
        # get the inputs 
        real_images_X[0], real_images_Y[0] = trainset[i]
        real_images_X = real_images_X.cuda()
        real_images_Y = real_images_Y.cuda()
        ones = torch.ones((batch_size,1,1,1))
        zeros = torch.zeros((batch_size,1,1,1))
        ones = ones.cuda()
        zeros = zeros.cuda()

        #F
        # zero the parameter gradients
        F_gen_optimizer.zero_grad()
        F_dis_optimizer.zero_grad()
        G_gen_optimizer.zero_grad()
        G_dis_optimizer.zero_grad()
        
        #Minimize discriminator loss against ones
        F_dis_real_output = F_dis(real_images_X)
        F_dis_real_loss = criteriondis(F_dis_real_output.float(), ones.float())
        F_dis_real_loss.backward()
        F_dis_optimizer.step()
        
        #Minimize discriminator loss against zeros
        F_gen_output = F_gen(real_images_X)
        F_dis_fake_output = F_dis(F_gen_output)
        F_dis_fake_loss = criteriondis(F_dis_fake_output.float(), zeros.float())
        F_dis_fake_loss.backward()
        F_dis_optimizer.step()
        
        #Minimize generator loss
        F_gen_output = F_gen(real_images_X)
        F_dis_fake_output = F_dis(F_gen_output)
        F_gen_loss = criteriondis(F_dis_fake_output.float(), ones.float())
        F_gen_loss.backward()
        F_gen_optimizer.step()
        
        #G
        #Minimize discriminator loss against ones
        G_dis_real_output = G_dis(real_images_Y)
        G_dis_real_loss = criteriondis(G_dis_real_output.float(), ones.float())
        G_dis_real_loss.backward()
        G_dis_optimizer.step()
        
        #Minimize discriminator loss against zeros
        G_gen_output = G_gen(real_images_Y)
        G_dis_fake_output = G_dis(G_gen_output)
        G_dis_fake_loss = criteriondis(G_dis_fake_output.float(), zeros.float())
        G_dis_fake_loss.backward()
        G_dis_optimizer.step()
        
        #Minimize generator loss
        G_gen_output = G_gen(real_images_Y)
        G_dis_fake_output = G_dis(G_gen_output)        
        G_gen_loss = criteriondis(G_dis_fake_output.float(), ones.float())
        G_gen_loss.backward()
        G_gen_optimizer.step()
        
        # zero the parameter gradients
        F_gen_optimizer.zero_grad()
        F_dis_optimizer.zero_grad()
        G_gen_optimizer.zero_grad()
        G_dis_optimizer.zero_grad()
        
        #Cycle consistency Loss
        # X > Y > X
        #F_gen_output = F_gen(real_images_X);
        G_output = G_gen(F_gen_output);
        F_cyclic_loss = criterion_feature(G_output, real_images_X);
        F_gen_optimizer.step();
        #G_gen_optimizer.step();
        
        # Y > X > Y
        #G_gen_output = G_gen(real_images_Y);
        F_output = F_gen(G_gen_output);
        G_cyclic_loss = criterion_feature(F_output, real_images_Y)
        G_gen_optimizer.step();
        #F_gen_optimizer.step();
        
        # print statistics
        F_running_loss_real += F_dis_real_loss.item()
        F_running_loss_fake += F_dis_fake_loss.item()
        F_running_loss_gen  += F_gen_loss.item()
        G_running_loss_real += G_dis_real_loss.item()
        G_running_loss_fake += G_dis_fake_loss.item()
        G_running_loss_gen  += G_gen_loss.item()
        F_running_cyclic_loss += F_cyclic_loss.item()
        G_running_cyclic_loss += G_cyclic_loss.item()
        
        i += 1;
        
        if i % 10 == 9:    # print every 2000 mini-batches
            print('Epoch: %d | No of images: %5d | F_real_loss: %.3f | F_fake_loss: %.3f | F_gen_loss: %.3f | F_cyclic_loss: %.3f' %
                  (epoch + 1, i + 1, F_running_loss_real / 10, F_running_loss_fake / 10, F_running_loss_gen / 10, F_running_cyclic_loss / 10))
            print('Epoch: %d | No of images: %5d | G_real_loss: %.3f | G_fake_loss: %.3f | G_gen_loss: %.3f | G_cyclic_loss: %.3f' %
                  (epoch + 1, i + 1, G_running_loss_real / 10, G_running_loss_fake / 10, G_running_loss_gen / 10, G_running_cyclic_loss / 10))
            F_running_loss_real = 0.0
            F_running_loss_fake = 0.0
            F_running_loss_gen  = 0.0
            G_running_loss_real = 0.0
            G_running_loss_fake = 0.0
            G_running_loss_gen  = 0.0
            F_running_cyclic_loss = 0.0
            G_running_cyclic_loss = 0.0 
#             Show output after every 2000 images
#             temp = gen_op.cpu()
#             imshow(torchvision.utils.make_grid(temp[0]));

print('Finished Training')

In [0]:
# #Check for memory
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()